In [6]:
import unittest
import time
import sys
import traceback
import requests

from bs4 import BeautifulSoup
from lxml import etree

#test
def list_chess_game(pyear):
    try:
        BASE_URL = "https://www.chessgames.com" 
        url = f"{BASE_URL}/perl/chess.pl?yearcomp=exactly&year="
        cnt = 1
        games = [] 
    
        time.sleep(3)
        tic = time.perf_counter()
        link = f"{url}{pyear}"
        reps = requests.get(link)
        toc = time.perf_counter()
            
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            soup = BeautifulSoup(html,'html.parser')
            tbs_parent = soup.find_all("table")
            tbs_child  = tbs_parent[1].find_all("table")
            rows   = tbs_child[1].find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                alla  = cols[0].find_all('a')
                for a in alla:
                    games.append(f"{BASE_URL}{a.attrs.get('href')}")
                                     
        return games          
    except ValueError:
            print(f"Error de Valor :{link}")
            return games                     
            #traceback.print_exception(*sys.exc_info())
    except:    
            print(f"Error general :{link}") 
            return games 
            #traceback.print_exception(*sys.exc_info())  

def Load_write_date(link):
    try:
       #carga el archivo de los datos
        file = open("chess_game.txt","a")
        gid = 1000000
        cnt = 1
        pgns = []
        
        # cargar la pagina
        time.sleep(3)
        tic = time.perf_counter()
        reps = requests.get(link)
        toc = time.perf_counter()
            
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            print(f"Process {toc - tic+1:0.4} seconds url: {link}")
            soup = BeautifulSoup(html,'html.parser')
            olga_data = soup.find_all("div", attrs={"id":"olga-data"})

            #Carga data
            if(olga_data): 
               pgn = olga_data[0].attrs.get('pgn')
               line = pgn.replace('\n',' ').replace('\r',' ')+'\n'
               file.write(line) 
            else :
              print("Error de contenido")

        file.close()    
    except ValueError:
            file.close()
            print("Error de Valor")
            traceback.print_exception(*sys.exc_info())
    except:    
            file.close()
            print("Error de general") 
            traceback.print_exception(*sys.exc_info())
year = 1794
year_end  = 1810
                                 
while(year <= year_end):
  gameslinks = list_chess_game(year)
  if(len(gameslinks)) > 0:                               
      for gamelink in gameslinks:            
          Load_write_date(gamelink)
  year = year + 1                              
print("Proceso finalizado")

Process 1.725 seconds url: https://www.chessgames.com/perl/chessgame?gid=1292807
Process 2.185 seconds url: https://www.chessgames.com/perl/chessgame?gid=1292808
Process 1.289 seconds url: https://www.chessgames.com/perl/chessgame?gid=1336216
Process 1.931 seconds url: https://www.chessgames.com/perl/chessgame?gid=1580880
Process 1.961 seconds url: https://www.chessgames.com/perl/chessgame?gid=1075430
Process 1.969 seconds url: https://www.chessgames.com/perl/chessgame?gid=1580891
Process 2.08 seconds url: https://www.chessgames.com/perl/chessgame?gid=1971676
Process 2.296 seconds url: https://www.chessgames.com/perl/chessgame?gid=1336477
Process 1.855 seconds url: https://www.chessgames.com/perl/chessgame?gid=1075431
Process 1.741 seconds url: https://www.chessgames.com/perl/chessgame?gid=1075432
Process 1.293 seconds url: https://www.chessgames.com/perl/chessgame?gid=1336228
Process 1.747 seconds url: https://www.chessgames.com/perl/chessgame?gid=1580833
Process 2.527 seconds url: ht